In [834]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.utils.np_utils import to_categorical

import matplotlib.pyplot as plt
import gradio as gr

from numpy import asarray
from numpy import zeros

import re
import json
import locale
import platform
from datetime import datetime
import numpy

if platform.system() == 'Linux':
    locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
elif platform.system() == 'Windows':
    locale.setlocale(locale.LC_TIME, 'Spanish_Spain.1252')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/enriquegomeztagle/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [835]:
# Lectura de formatos .json para entrenar cada modelo y asignación
# de información correspondiente
with open('Intenciones_NivelI.json', encoding='utf-8') as file:
    data_NivelI = json.load(file)

with open('Intenciones_NivelIIA.json', encoding='utf-8') as file:
    data_NivelIIA = json.load(file)

with open('Intenciones_NivelIIB.json', encoding='utf-8') as file:
    data_NivelIIB = json.load(file)


In [836]:
NI = dict()
NIIA = dict()
NIIB = dict()

for info in data_NivelI['intents']:
    NI.setdefault(info['tag'], info['patterns'])

for info in data_NivelIIA['intents']:
    NIIA.setdefault(info['tag'], info['patterns'])

for info in data_NivelIIB['intents']:
    NIIB.setdefault(info['tag'], info['patterns'])

# print(NI)
# print(NIIA)
# print(NIIB)


In [837]:
Y_NI = list()
Y_NIIA = list()
Y_NIIB = list()

for clase, lista_textos in NI.items():
    for text in lista_textos:
        Y_NI.append(list(NI.keys()).index(clase))

for clase, lista_textos in NIIA.items():
    for text in lista_textos:
        Y_NIIA.append(list(NIIA.keys()).index(clase))

for clase, lista_textos in NIIB.items():
    for text in lista_textos:
        Y_NIIB.append(list(NIIB.keys()).index(clase))

# print("Vector de salidas Y para N1:")
# print(Y_NI)
# print("Vector de salidas Y para N2A:")
# print(Y_NIIA)
# print("Vector de salidas Y para N2B:")
# print(Y_NIIB)


In [838]:
def quitar_stopwords(Textos):
    X = list()
    for sentence in Textos:
        for stopword in stop_words:
            sentence = sentence.replace(" " + stopword + " ", " ")
        sentence = sentence.replace("á", "a")
        sentence = sentence.replace("é", "e")
        sentence = sentence.replace("í", "i")
        sentence = sentence.replace("ó", "o")
        sentence = sentence.replace("ú", "u")

        sentence = re.sub(r'\s+', ' ', sentence)
        sentence = sentence.lower()
        tokenizer = RegexpTokenizer(r'\w+')
        result = tokenizer.tokenize(sentence)
        X.append(TreebankWordDetokenizer().detokenize(result))
    return X


In [839]:
Textos_NI = list()
for Lista in NI.values():
    for Texto in Lista:
        Textos_NI.append(Texto)

X_NI = quitar_stopwords(Textos_NI)

Textos_NIIA = list()
for Lista in NIIA.values():
    for Texto in Lista:
        Textos_NIIA.append(Texto)

X_NIIA = quitar_stopwords(Textos_NIIA)

Textos_NIIB = list()
for Lista in NIIB.values():
    for Texto in Lista:
        Textos_NIIB.append(Texto)

X_NIIB = quitar_stopwords(Textos_NIIB)


In [840]:
# print(X_NI)
# print(X_NIIA)
# print(X_NIIB)


In [841]:
maxlen = 5

tokenizer_NI = Tokenizer(num_words=5000)
tokenizer_NIIA = Tokenizer(num_words=5000)
tokenizer_NIIB = Tokenizer(num_words=5000)

tokenizer_NI.fit_on_texts(X_NI)
X_NI_Tok = tokenizer_NI.texts_to_sequences(X_NI)
tokenizer_NIIA.fit_on_texts(X_NIIA)
X_NIIA_Tok = tokenizer_NIIA.texts_to_sequences(X_NIIA)
tokenizer_NIIB.fit_on_texts(X_NIIB)
X_NIIB_Tok = tokenizer_NIIB.texts_to_sequences(X_NIIB)


X_NI_train = pad_sequences(X_NI_Tok, padding='post', maxlen=maxlen)
X_NIIA_train = pad_sequences(X_NIIA_Tok, padding='post', maxlen=maxlen)
X_NIIB_train = pad_sequences(X_NIIB_Tok, padding='post', maxlen=maxlen)


In [842]:
# print("Matriz de entrada para NI:")
# print(X_NI_train)


In [843]:
# print("Matriz de entrada para NIIA:")
# print(X_NIIA_train)


In [844]:
# print("Matriz de entrada para NIIB:")
# print(X_NIIB_train)


In [845]:
embeddings_dictionary = dict()
Embeddings_file = open('Word2Vect_Spanish.txt', encoding="utf8")

for linea in Embeddings_file:
    caracts = linea.split()
    palabra = caracts[0]
    vector = asarray(caracts[1:], dtype='float32')
    embeddings_dictionary[palabra] = vector
Embeddings_file.close()


In [846]:
def Asignar_Embeddings(tokenizer, vocab_size):
    embedding_matrix = zeros((vocab_size, 300))
    for word, index in tokenizer.word_index.items():
        embedding_vector = embeddings_dictionary.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
    return embedding_matrix


In [847]:
vocab_size_NI = len(tokenizer_NI.word_index) + 1
embedding_matrix_NI = Asignar_Embeddings(tokenizer_NI, vocab_size_NI)

vocab_size_NIIA = len(tokenizer_NIIA.word_index) + 1
embedding_matrix_NIIA = Asignar_Embeddings(tokenizer_NIIA, vocab_size_NIIA)

vocab_size_NIIB = len(tokenizer_NIIB.word_index) + 1
embedding_matrix_NIIB = Asignar_Embeddings(tokenizer_NIIB, vocab_size_NIIB)


In [848]:
def Definir_Modelos(vocab_size, embedding_matrix, X_train, labels):
    model = Sequential()
    embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=X_train.shape[1],
                                trainable=False)
    model.add(embedding_layer)
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(len(labels), activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # print(model.summary())

    # print("\nPalabras en el vocabulario:")
    # print(vocab_size)
    return model


In [849]:
model_NI = Definir_Modelos(vocab_size_NI, embedding_matrix_NI, X_NI_train, NI.keys())


In [850]:
model_NIIA = Definir_Modelos(vocab_size_NIIA, embedding_matrix_NIIA, X_NIIA_train, NIIA.keys())


In [851]:
model_NIIB = Definir_Modelos(vocab_size_NIIB, embedding_matrix_NIIB, X_NIIB_train, NIIB.keys())


In [852]:
def Entrenar_Modelos(X_train, Y, model, labels):
    train_labels = to_categorical(Y, num_classes=len(labels))
    # print('Matriz de salidas')
    # print(train_labels)

    history = model.fit(X_train, train_labels, epochs=30, batch_size=1, verbose=1)

    score = model.evaluate(X_train, train_labels, verbose=1)
    # print("\nTest Loss:", score[0])
    # print("Test Accuracy:", score[1])
    return history


In [853]:
history_NI = Entrenar_Modelos(X_NI_train, Y_NI, model_NI, NI.keys())


Epoch 1/30
71/71 [==============================] - 3s 4ms/step - loss: 1.4842 - accuracy: 0.2676
Epoch 2/30
71/71 [==============================] - 0s 4ms/step - loss: 0.9092 - accuracy: 0.6056
Epoch 3/30
71/71 [==============================] - 0s 5ms/step - loss: 0.4686 - accuracy: 0.8169
Epoch 4/30
71/71 [==============================] - 0s 5ms/step - loss: 0.1912 - accuracy: 0.9437
Epoch 5/30
71/71 [==============================] - 0s 5ms/step - loss: 0.1003 - accuracy: 0.9859
Epoch 6/30
71/71 [==============================] - 0s 5ms/step - loss: 0.0522 - accuracy: 0.9859
Epoch 7/30
71/71 [==============================] - 0s 5ms/step - loss: 0.1284 - accuracy: 0.9437
Epoch 8/30
71/71 [==============================] - 0s 5ms/step - loss: 0.1532 - accuracy: 0.9577
Epoch 9/30
71/71 [==============================] - 0s 5ms/step - loss: 0.0258 - accuracy: 1.0000
Epoch 10/30
71/71 [==============================] - 0s 5ms/step - loss: 0.0133 - accuracy: 1.0000
Epoch 11/30
71/71 [

In [854]:
history_NIIA = Entrenar_Modelos(X_NIIA_train, Y_NIIA, model_NIIA, NIIA.keys())


Epoch 1/30


55/55 [==============================] - 1s 4ms/step - loss: 1.2960 - accuracy: 0.5091
Epoch 2/30
55/55 [==============================] - 0s 4ms/step - loss: 0.7068 - accuracy: 0.8000
Epoch 3/30
55/55 [==============================] - 0s 5ms/step - loss: 0.2594 - accuracy: 0.9273
Epoch 4/30
55/55 [==============================] - 0s 5ms/step - loss: 0.0972 - accuracy: 1.0000
Epoch 5/30
55/55 [==============================] - 0s 5ms/step - loss: 0.0288 - accuracy: 1.0000
Epoch 6/30
55/55 [==============================] - 0s 5ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 7/30
55/55 [==============================] - 0s 5ms/step - loss: 0.1175 - accuracy: 0.9818
Epoch 8/30
55/55 [==============================] - 0s 5ms/step - loss: 0.3514 - accuracy: 0.9273
Epoch 9/30
55/55 [==============================] - 0s 5ms/step - loss: 0.0124 - accuracy: 1.0000
Epoch 10/30
55/55 [==============================] - 0s 5ms/step - loss: 0.0081 - accuracy: 1.0000
Epoch 11/30
55/55 [===========

In [855]:
history_NIIB = Entrenar_Modelos(X_NIIB_train, Y_NIIB, model_NIIB, NIIB.keys())


Epoch 1/30


22/22 [==============================] - 1s 3ms/step - loss: 1.0909 - accuracy: 0.3182
Epoch 2/30
22/22 [==============================] - 0s 4ms/step - loss: 0.9375 - accuracy: 0.4091
Epoch 3/30
22/22 [==============================] - 0s 4ms/step - loss: 0.7658 - accuracy: 0.5000
Epoch 4/30
22/22 [==============================] - 0s 5ms/step - loss: 0.5090 - accuracy: 0.9091
Epoch 5/30
22/22 [==============================] - 0s 5ms/step - loss: 0.1993 - accuracy: 1.0000
Epoch 6/30
22/22 [==============================] - 0s 5ms/step - loss: 0.0406 - accuracy: 1.0000
Epoch 7/30
22/22 [==============================] - 0s 5ms/step - loss: 0.0280 - accuracy: 1.0000
Epoch 8/30
22/22 [==============================] - 0s 5ms/step - loss: 0.0414 - accuracy: 1.0000
Epoch 9/30
22/22 [==============================] - 0s 5ms/step - loss: 0.0109 - accuracy: 1.0000
Epoch 10/30
22/22 [==============================] - 0s 5ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 11/30
22/22 [===========

In [856]:
def Grafica_Modelo(history):
    plt.figure(figsize=(12, 5))
    plt.ylim(-0.1, 1.1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['loss'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Acc', 'Loss'])
    plt.show()


In [857]:
# Grafica_Modelo(history_NI)


In [858]:
# Grafica_Modelo(history_NIIA)


In [859]:
# Grafica_Modelo(history_NIIB)


# Programación del Árbol conversacional

In [860]:
def instancer(inp, model, tags):
    inp = inp.lower().replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o")
    inp = inp.replace("ú", "u").replace("¿", "").replace("?", "")
    txt = [inp]
    seq = tokenizer_NI.texts_to_sequences(txt)
    padded = pad_sequences(seq, maxlen=maxlen)

    # print("Input shape before predict:", padded.shape)
    # print("Input data:", padded)

    results = model.predict(padded)
    results_index = numpy.argmax(results)
    tag = list(tags.keys())[results_index]
    maxscore = numpy.max(results)
    return tag, maxscore


In [861]:
global estado_actual
global Tipo_Cuarto
global Fecha_entrada
global Fecha_salida
global Num_huespedes
global Servicio_deseado
global Espacio_Apartado
global Servicio_Pedido
global Comida_Pedida


In [862]:

def Activar_NI(user_input):
    if user_input is None:
        response = ""
        return response, 'NI'

    if user_input == "AUTORES":
            response = "Easter Egg: Elaborado por Mau y Kike"
            return response, 'NI'
    
    if user_input == "MEJOR PROFE":
            response = "Easter Egg: IRVING"
            return response, 'NI'
    
    if user_input == "SALUDOS A MICH":
            response = "Easter Egg: Saludos Miich, un gusto verte por acá"
            return response, 'NI'
    
    tag, maxscore = instancer(user_input, model_NI, NI)

    if maxscore > 0.65 or user_input == 'salir':
        if user_input == 'salir':
            response = "Hasta luego, fue un gusto hablar contigo"
            return response, 'salir'

        if tag == 'Reservar_Habitacion':
            next_state = 'NIIA1'
            response = "Entendido, estás interesado en reservar una habitación. ¿Qué tipo de habitación deseas?"

        elif tag == 'Servicio_hotel':
            if user_input.count('Apartar_espacio') > 0:
                next_state = 'NIIB1'
            elif user_input.count('Servicio') > 0:
                next_state = 'NIIB2'
            elif user_input.count('Comida_bebida') > 0:
                next_state = 'NIIB3'
            else:
                next_state = 'NIIB'
            response = "Claro, puedo ayudarte con los servicios del hotel. (pedir comida, apartar espacio,...)"

        elif tag == 'Historia_Hotel':
            response = "Desde su apertura en 1920 por el pionero Don Eduardo Mendoza, nuestro hotel es un ícono de la arquitectura Art Deco, diseñado por Carlos Fontana.\nHa acogido a luminarias como la actriz Clara Estrella en los años 30 y fue el lugar de la histórica cumbre de paz en 1955. Cada habitación cuenta una historia; por ejemplo, la suite 204, donde el novelista Luis Montero escribió su obra maestra. Es un lugar donde la historia y la hospitalidad se encuentran en cada esquina."
            next_state = 'NI'

        elif tag == 'Atracciones_Cercanas':
            response = "En el corazón de la ciudad, nuestro hotel se encuentra a un corto trayecto del Parque Central y a unos minutos del Museo Nacional de Arte, perfecto para los entusiastas de la cultura. Para aquellos interesados en la historia, el Convento de San Francisco está a una caminata de distancia. No te pierdas el Mercado del Sol, ideal para saborear la cocina local y encontrar artesanías únicas. Además, ofrecemos excursiones al Castillo de San Lorenzo, un sitio histórico con vistas espectaculares, a solo media hora en coche."
            next_state = 'NI'

        elif tag == "Famosos_hotel":
            response = "Nuestro hotel ha sido un destino favorito para muchas personalidades destacadas a lo largo de los años. Entre los huéspedes ilustres, hemos tenido el placer de acoger a figuras como el actor ganador del Oscar Leonardo DiCaprio, la aclamada cantante Adele, y el empresario tecnológico Elon Musk. Además, leyendas del deporte como Serena Williams y estrellas de la talla de Beyoncé también han disfrutado de la exclusividad y el servicio de primera clase de nuestro establecimiento. Cada celebridad que nos visita añade un capítulo único a la rica historia de nuestro hotel, asegurando que no solo sea un lugar de lujo, sino también un espacio donde se cruzan caminos extraordinarios"
            next_state = 'NI'

        else:
            next_state = 'NI'
            response = "Puedes preguntarme sobre reservaciones, servicios del hotel, su historia o atracciones cercanas."

        return response, next_state

    else:
        response = "Lo siento, pero no entendí tu petición, ¿Podrías decirlo de otra forma?"
        return response, 'NI'


In [874]:
def Activar_NIIA(user_input):
    if user_input is None:
        response = "Escribe 'reservar habitación' para confirmar"
        return response, 'NIIA'

    tag, maxscore = instancer(user_input, model_NIIA, NIIA)

    if maxscore > 0.2 or user_input == 'salir' or user_input == 'volver':
        if user_input == 'volver':
            next_state = 'NI'
            response = "Entendido, volviendo al menú principal."
        elif user_input == 'salir':
            response = "Hasta luego, fue un gusto hablar contigo."
            return response
        else:
            next_state = 'NIIA1'        

        return response, next_state
    else:
        response = "Lo siento, pero no entendí.. Puedes internatar de nuevo"
        
        return response, 'NIIA'


In [864]:
def Activar_NIIA1(user_input=None):
    if user_input is None:
        response = ""
        return response, 'NIIA1'
    
    user_input = user_input.lower()

    tag, maxscore = instancer(user_input, model_NIIA, NIIA)

    if maxscore > 0.5 or user_input == 'salir' or user_input == 'volver':
        if user_input == 'volver':
            next_state = 'NI'
            response = "Entendido, volviendo al menú principal."
        elif user_input == 'salir':
            response = "Hasta luego, fue un gusto hablar contigo."
            return response
        elif 'estándar' in user_input or 'económica' in user_input:
            response = "Excelente, reservaremos una habitación estándar. ¿Para cuándo agendamos su entrada? (dd/mm/aaaa)"
            Tipo_Cuarto = "Estándar"
            next_state = 'NIIA2'
        elif 'suite' in user_input:
            response = "Excelente, reservaremos una habitación suite. ¿Para cuándo agendamos su entrada? (dd/mm/aaaa)"
            Tipo_Cuarto = "Suite"
            next_state = 'NIIA2'
        elif 'doble' in user_input or 'dos camas' in user_input:
            response = "Excelente, reservaremos una habitación doble. ¿Para cuándo agendamos su entrada? (dd/mm/aaaa)"
            Tipo_Cuarto = "Doble"
            next_state = 'NIIA2'
        elif 'familiar' in user_input:
            response = "Excelente, reservaremos una habitación familiar. ¿Para cuándo agendamos su entrada? (dd/mm/aaaa)"
            Tipo_Cuarto = "Familiar"
            next_state = 'NIIA2'
        elif 'deluxe' in user_input:
            response = "Excelente, reservaremos una habitación deluxe. ¿Para cuándo agendamos su entrada? (dd/mm/aaaa)"
            Tipo_Cuarto = "Deluxe"
            next_state = 'NIIA2'
        elif 'ofrecen' in user_input:
            response = "Ofrecemos los siguientes tipos de habitaciones: estándar, doble, familiar, suite, y deluxe."
        elif user_input == 'volver':
            next_state = 'NIIA'
            response = "De acuerdo, volviendo al menú anterior."
        elif user_input == 'salir':
            next_state = 'salir'
            response = "Hasta luego, fue un gusto ayudarte."
        else:
            next_state = 'NIIA1'
            response = "No he podido identificar el tipo de habitación. ¿Puedes especificar si deseas una habitación estándar, suite, doble, etc.?"

        return response, next_state


In [865]:
def Activar_NIIA2(user_input):
    if user_input is None:
        response = ""

    user_input = user_input.lower()

    if user_input == 'volver':
        next_state = 'NI'
        response = "De acuerdo, volviendo al menú anterior."
    elif user_input == 'salir':
        response = "Hasta luego, fue un gusto hablar contigo."
        next_state = 'salir'
    else:
        try:
            fecha_entrada_usuario = datetime.strptime(user_input, '%d/%m/%Y')
            if fecha_entrada_usuario.date() < datetime.now().date():
                response = "No puedes agendar una fecha en el pasado. Por favor, elige una fecha futura."
                next_state = 'NIIA2'
            else:
                fecha_formateada = fecha_entrada_usuario.strftime('%d de %B del %Y')
                response = f"Muy bien, agendaré la entrada para el {fecha_formateada}. ¿Para cuándo agendamos su salida? (dd/mm/aaaa)"
                Fecha_entrada = fecha_entrada_usuario
                next_state = 'NIIA3'
        except ValueError:
            response = "Parece que la fecha no está en el formato correcto. Por favor, ingresa la fecha en formato dd/mm/aaaa."
            next_state = 'NIIA2'

    return response, next_state


In [878]:
def Activar_NIIA3(user_input):
    if user_input is None:
        response = ""

    user_input = user_input.lower()

    if user_input == 'volver':
        next_state = 'NI'
        response = "De acuerdo, volviendo al menú anterior."
    elif user_input == 'salir':
        response = "Hasta luego, fue un gusto hablar contigo."
        next_state = 'salir'
    else:
        try:
            fecha_salida_usuario = datetime.strptime(user_input, '%d/%m/%Y')
            if fecha_salida_usuario.date() < datetime.now().date():
                response = "No puedes agendar una fecha de salida en el pasado. Por favor, elige una fecha futura."
            else:
                fecha_formateada = fecha_salida_usuario.strftime('%d de %B del %Y')
                response = f"Muy bien, agendaré la salida para el {fecha_formateada}. ¿Para cuántos huéspedes?"
                Fecha_salida = fecha_salida_usuario
                next_state = 'NIIA4'
        except ValueError:
            response = "Parece que la fecha no está en el formato correcto. Por favor, ingresa la fecha en formato dd/mm/aaaa."
            next_state = 'NIIA3'

    return response, next_state


In [867]:
def Activar_NIIA4(user_input):
    if user_input is None:
        response = ""
        return response, 'NI'

    user_input = user_input.lower()

    if user_input == 'volver':
        next_state = 'NI'
        response = "De acuerdo, volviendo al menú anterior."
    elif user_input == 'salir':
        response = "Hasta luego, fue un gusto hablar contigo."
        next_state = 'salir'
    else:
        response = "Reservación completada.. Presiona 'CLEAR' para continuar"
        next_state = 'NI'

    return response, next_state


In [880]:
def Activar_NIIB(user_input=None):
    if user_input is None:
        response = ""

    tag, maxscore = instancer(user_input, model_NIIB, NIIB)

    if maxscore > 0.2 or user_input == 'salir' or user_input == 'volver':
        if user_input == 'volver':
            next_state = 'NI'
            response = "De acuerdo, volviendo al menú anterior."
        elif user_input == 'salir':
            response = "Hasta luego, fue un gusto hablar contigo."
            next_state = 'salir'
        elif tag == 'Apartar_espacio':
            next_state = 'NIIB1'
            response = "Excelente! Apartemos un espacio. ¿Qué tipo de espacio te gustaría reservar?"
        elif tag == 'Servicio':
            next_state = 'NIIB2'
            response = "Con gusto! ¿Qué servicio le gustaría pedir"
        elif tag == 'Comida_bebida':
            next_state = 'NIIB3'
            response = "Hora de comer! ¿Qué alimento o bebida le gustaría pedir?"
        else:
            response = "No estoy seguro de cómo ayudarte con eso. ¿Podrías especificar más tu solicitud?"
            next_state = 'NIIB'
    else:
        response = "Lo siento, pero no entendí. ¿Podrías ser más específico?"
        next_state = 'NIIB'

    return response, next_state


In [869]:
def Activar_NIIB1(user_input):
    if user_input is None:
        response = "Creo que no dijiste nada, ¿Qué tipo de espacio te gustaría reservar?"
        next_state = 'NIIB1'
        return response, next_state

    user_input = user_input.lower()

    if user_input == 'volver':
        next_state = 'NI'
        response = "De acuerdo, volviendo al menú anterior."
    elif user_input == 'salir':
        response = "Hasta luego, fue un gusto hablar contigo."
        next_state = 'salir'
    elif 'alberca' in user_input:
        response = "Excelente, reservaremos la alberca. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Espacio_Apartado = "alberca"
        next_state = 'NI'
    elif 'gimnasio' in user_input:
        response = "Excelente, reservaremos el gimnasio. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Espacio_Apartado = "gimnasio"
        next_state = 'NI'
    elif 'eventos' in user_input or 'salon' in user_input:
        response = "Excelente, reservaremos el salón de eventos. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Espacio_Apartado = "salón de eventos"
        next_state = 'NI'
    elif 'terraza' in user_input:
        response = "Excelente, reservaremos la terraza. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Espacio_Apartado = "terraza"
        next_state = 'NI'
    elif 'spa' in user_input:
        response = "Excelente, reservaremos el spa. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Espacio_Apartado = "spa"
        next_state = 'NI'
    else:
        response = "¿Podrías repetirme qué tipo de espacio te gustaría reservar?"
        next_state = 'NIIB1'

    return response, next_state


In [870]:
def Activar_NIIB2(user_input=None):
    if user_input is None:
        response = "Creo que no dijiste nada, ¿Qué servicio le gustaría pedir?"
        next_state = 'NIIB2'
        return response, next_state

    user_input = user_input.lower()

    if user_input == 'volver':
        next_state = 'NI'
        response = "De acuerdo, volviendo al menú anterior."
    elif user_input == 'salir':
        response = "Hasta luego, fue un gusto hablar contigo."
        next_state = 'salir'
    elif user_input.count('limpiar') > 0:
        response = "Claro, enviaremos a alguien para limpiar la habitación. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Servicio_Pedido = "limpieza de cuarto"
        next_state = 'NI'
    elif user_input.count('toallas') > 0:
        response = "Ok, enviaremos toallas a su habitación. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Servicio_Pedido = "toallas"
        next_state = 'NI'
    elif user_input.count('jabón') > 0 or user_input.count('shampoo') > 0:
        response = "Seguro! enviaremos artículos de higiene personal a su cuarto. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Servicio_Pedido = "higiene personal"
        next_state = 'NI'
    elif user_input.count('equipaje') > 0 or user_input.count('maletas') > 0:
        response = "Mandaremos a alguien para apoyar con el equipaje. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Servicio_Pedido = "apoyo con equipaje"
        next_state = 'NI'
    elif user_input.count('almohadas') > 0:
        response = "Con gusto enviaremos más almohadas a la habitación. Si hay otro servicio que te gustaría pedir no dudes en decírmelo!"
        Servicio_Pedido = "almohadas"
        next_state = 'NI'
    else:
        response = "¿Podrías repetirme qué servicio requieres?"
        next_state = 'NIIB2'

    return response, next_state


In [871]:
def Activar_NIIB3(user_input=None):
    if user_input is None:
        response = "Creo que no dijiste nada, ¿Qué alimento o bebida le gustaría pedir?"
        next_state = 'NIIB3'
        return response, next_state

    user_input = user_input.lower()

    if user_input == 'volver':
        next_state = 'NI'
        response = "De acuerdo, volviendo al menú anterior."
    elif user_input == 'salir':
        response = "Hasta luego, fue un gusto hablar contigo."
        next_state = 'salir'
    elif user_input.count('desayuno') > 0:
        response = "Claro, enviaremos el desayuno a la habitación."
        comida_pedida = "desayuno"
        next_state = 'NI'
    elif user_input.count('vegana') > 0 or user_input.count('vegetariana') > 0:
        response = "Por supuesto, prepararemos una opción vegana/vegetariana para usted."
        comida_pedida = "vegana/vegetariana"
        next_state = 'NI'
    elif user_input.count('comida') > 0:
        response = "Mandaremos la comida a la habitación."
        comida_pedida = "comida"
        next_state = 'NI'
    elif user_input.count('botella') > 0 or user_input.count('vino') > 0:
        response = "Seguro! enviaremos una botella de vino."
        comida_pedida = "vino"
        next_state = 'NI'
    elif user_input.count('café') > 0:
        response = "Con gusto enviaremos café a su habitación."
        comida_pedida = "café"
        next_state = 'NI'
    elif user_input.count('romántica') > 0:
        response = "Prepararemos una cena romántica para usted."
        comida_pedida = "romántica"
        next_state = 'NI'
    elif user_input.count('cena') > 0:
        response = "Claro, mandaremos la cena a la habitación."
        comida_pedida = "cena"
        next_state = 'NI'
    else:
        response = "¿Podrías repetirme qué alimento/bebida quieres pedir?"
        next_state = 'NIIB3'

    return response, next_state


In [881]:
# Implementación de casos correspondientes para cada nivel del ChatBot
# Nivel contextual inicial por defecto, el primero


estado_actual = "NI"

maquina_estados = {'NI': Activar_NI,
                   "NIIA": Activar_NIIA,
                   "NIIA1": Activar_NIIA1,
                   "NIIA2": Activar_NIIA2,
                   "NIIA3": Activar_NIIA3,
                   "NIIA4": Activar_NIIA4,
                   "NIIB": Activar_NIIB,
                   "NIIB1": Activar_NIIB1,
                   "NIIB2": Activar_NIIB2,
                   "NIIB3": Activar_NIIB3
                   }

def adapted_chat1(user_input, Nivel):
    if user_input == "":
        response, next_state = maquina_estados[Nivel](None)
    else:
        response, next_state = maquina_estados[Nivel](user_input)
    return response, next_state

def gradio_chatbot_function(user_input, state='NI'):
    if user_input == "":
        response, next_state = adapted_chat1(None, state)
    else:
        response, next_state = adapted_chat1(user_input, state)
    return response, next_state

iface = gr.Interface(
    fn=gradio_chatbot_function,
    inputs=[gr.Textbox(lines=2, label="Tu mensaje", placeholder="Escribe tu mensaje aquí..."), "state"],
    outputs=[gr.Textbox(label="Respuesta del Chatbot", placeholder="Puedes preguntarme sobre reservaciones, servicios del hotel, su historia o atracciones cercanas"), "state"],
    title="Hotel ESDAI Chatbot",
    description="Bienvenido a Hotel ESDAI Chatbot. ¿Qué deseas hacer?",
    allow_flagging='never'
)

iface.launch()


Running on local URL:  http://127.0.0.1:7900

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 23ms/step


2023-11-07 02:25:30.441197: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: indices[0,4] = 31 is not in [0, 30)
	 [[{{node sequential_59/embedding_59/embedding_lookup}}]]
Traceback (most recent call last):
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/routes.py", line 534, in predict
    output = await route_utils.call_process_api(
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/route_utils.py", line 226, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/blocks.py", line 1550, in process_api
    result = await self.call_function(
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/blocks.py", line 11

1/1 [==============================] - 0s 18ms/step


2023-11-07 02:26:14.936578: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: indices[0,4] = 31 is not in [0, 30)
	 [[{{node sequential_59/embedding_59/embedding_lookup}}]]
Traceback (most recent call last):
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/routes.py", line 534, in predict
    output = await route_utils.call_process_api(
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/route_utils.py", line 226, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/blocks.py", line 1550, in process_api
    result = await self.call_function(
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/blocks.py", line 11

1/1 [==============================] - 0s 17ms/step


In [873]:
# # Check the size of the tokenizer's word index
# print("Tokenizer word index size:", len(tokenizer_NI.word_index))
# 
# # After loading the model, check the input dimension of the embedding layer
# embedding_input_dim = model_NIIB.layers[0].get_config()['input_dim']
# print("Model's embedding input dimension:", embedding_input_dim)


1/1 [==============================] - 0s 18ms/step


Traceback (most recent call last):
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/routes.py", line 534, in predict
    output = await route_utils.call_process_api(
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/route_utils.py", line 226, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/blocks.py", line 1550, in process_api
    result = await self.call_function(
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/gradio/blocks.py", line 1185, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib/python3.9/site-packages/anyio/to_thread.py", line 28, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(func, *args, cancellable=cancellable,
  File "/Users/enriquegomeztagle/anaconda3/envs/NLP/lib

1/1 [==============================] - 0s 23ms/step
